In [ ]:
!pip install -q optuna cmaes

In [ ]:
#for using GPSampler
!pip install -q scipy torch

In [ ]:
import numpy as np

def ackley_2d(x, y, a=20, b=0.2, c=2*np.pi):
    return -a * np.exp(-b * np.sqrt(0.5 * (x**2 + y**2))) \
        - np.exp(0.5 * (np.cos(c*x) + np.cos(c*y))) \
        + a \
        + np.exp(1)

In [ ]:
import optuna
import matplotlib.pyplot as plt

optuna.logging.set_verbosity(optuna.logging.WARNING)


# 目标函数 (使用上面定义的 ackley_2d)
def objective(trial):
    x = trial.suggest_float('x', -5, 5) # 定义 x 的搜索范围
    y = trial.suggest_float('y', -5, 5) # 定义 y 的搜索范围
    return ackley_2d(x, y)

# 要比较的 Sampler
samplers = {
    "GridSampler": optuna.samplers.GridSampler(search_space={
        "x": np.linspace(-5, 5, 10).tolist(), 
        "y": np.linspace(-5, 5, 10).tolist()
    }), # GridSampler 在这里可能不太直观展示探索过程
    "RandomSampler": optuna.samplers.RandomSampler(),
    "TPESampler": optuna.samplers.TPESampler(),
    "CmaEsSampler": optuna.samplers.CmaEsSampler(),
}

# 存储每个 Sampler 的采样点
sampled_points = {}

# 运行优化并记录采样点
for sampler_name, sampler in samplers.items():
    study = optuna.create_study(sampler=sampler)
    study.optimize(objective, n_trials=100, show_progress_bar=True) # 运行 100 次 trials，不显示进度条
    trials = study.trials
    points_x = [t.params['x'] for t in trials] # 提取 x 坐标
    points_y = [t.params['y'] for t in trials] # 提取 y 坐标
    sampled_points[sampler_name] = (points_x, points_y) # 存储采样点

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D



x = np.linspace(-5, 5, 1000)
y = np.linspace(-5, 5, 1000)
X, Y = np.meshgrid(x, y)
Z = ackley_2d(X, Y)


# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z, cmap='viridis')

# Set labels and title
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z (Ackley)')

# # Adjust view angle and margins
# ax.view_init(elev=30, azim=45)  # Adjust elevation and azimuth
# plt.subplots_adjust(left=1, right=2, bottom=1, top=2) # Adjust margins

ax.set_box_aspect(aspect=None, zoom=0.9) # Adjust the zoom value

# Show the plot
plt.show()



# ---------

plt.figure(figsize=(10, 8))

# 绘制 Ackley 函数的等高线图
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)
Z = ackley_2d(X, Y)
contour = plt.contourf(X, Y, Z, levels=20, cmap='viridis') # levels 参数控制等高线密度，cmap 选择颜色映射
plt.colorbar(label='Ackley Value')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Ackley Function Heatmap')
plt.show()


In [ ]:
# 可视化
plt.figure(figsize=(10, 8))

# 绘制 Ackley 函数的等高线图
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)
Z = ackley_2d(X, Y)
contour = plt.contourf(X, Y, Z, levels=20, cmap='viridis') # levels 参数控制等高线密度，cmap 选择颜色映射
plt.colorbar(contour, label="Objective Value") # 显示颜色条

# 叠加不同 Sampler 的采样点
colors = ['red', 'blue', 'white', 'orange'] # 为不同 Sampler 设置颜色
markers = ['D', 'P', 'x', '*'] # 为不同 Sampler 设置标记
for i, (sampler_name, points) in enumerate(sampled_points.items()):
    plt.scatter(points[0], points[1], label=sampler_name, color=colors[i], marker=markers[i], s=20) # 绘制散点图，s 参数控制点的大小

plt.title("Comparison of Optuna Samplers on Ackley 2D Function") # 图表标题
plt.xlabel("x") # x 轴标签
plt.ylabel("y") # y 轴标签
plt.legend() # 显示图例
plt.grid(False) # 可以选择不显示网格线，使图更简洁
plt.show()

In [ ]:
import optuna
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.cm as cm # 导入 colormap 模块


# 1. 目标函数 (Ackley 2D)
def ackley_2d(x, y, a=20, b=0.2, c=2*np.pi):
    return -a * np.exp(-b * np.sqrt(0.5 * (x**2 + y**2))) - np.exp(0.5 * (np.cos(c*x) + np.cos(c*y))) + a + np.exp(1)

# 2. 要比较的 Sampler
samplers = {
    # "QMCSampler": optuna.samplers.QMCSampler(),
    # "NSGAIISampler": optuna.samplers.NSGAIISampler(),
    # "GPSampler": optuna.samplers.GPSampler(),
    # "PartialFixedSampler": optuna.samplers.PartialFixedSampler({
    #     "y": 0,
    # }, optuna.samplers.GPSampler()),
    # "RandomSampler": optuna.samplers.RandomSampler(),
    "TPESampler": optuna.samplers.TPESampler(),
    # "CmaEsSampler": optuna.samplers.CmaEsSampler(),
    # "GridSampler": optuna.samplers.GridSampler(search_space={
    #     "x": np.linspace(-5, 5, 10).tolist(), 
    #     "y": np.linspace(-5, 5, 10).tolist()
    # }), # GridSampler 在这里可能不太直观展示探索过程
}

# color_map_options = ['viridis', 'plasma', 'coolwarm', 'spring', 'cool', 'hot', 'Wistia', 'winter', 'autumn'] # 尝试不同的颜色映射
color_map_options = ['autumn'] # 尝试不同的颜色映射
n_trails_tries = [100, 200, 500, 1000]

# 3. 优化目标函数
def objective(trial):
    x = trial.suggest_float('x', -5, 5)
    y = trial.suggest_float('y', -5, 5)
    return ackley_2d(x, y)


# 4. 动画帧生成函数
def create_frame(trial_number, study, sampler_name, best_value_so_far, contour_levels=20, cmap_contour='viridis', cmap_points='viridis'): # Add best_value_so_far parameter
    """生成动画的单帧图像"""
    plt.clf() # 清除当前 axes，准备绘制新帧

    # 绘制 Ackley 函数等高线
    x_range = np.linspace(-5, 5, 100)
    y_range = np.linspace(-5, 5, 100)
    X, Y = np.meshgrid(x_range, y_range)
    Z = ackley_2d(X, Y)
    contour = plt.contourf(X, Y, Z, levels=contour_levels, cmap=cmap_contour)
    plt.colorbar(contour, label="Objective Value")

    
    # 获取到当前 trial_number 为止的 trials
    trials = study.trials[:trial_number]
    points_x = []
    points_y = []
    trial_indices = [] # 存储 trial 的索引，用于颜色映射

    for i, trial in enumerate(trials): # 遍历 trials 时，同时获取索引 i
        if trial.state == optuna.trial.TrialState.COMPLETE:
            points_x.append(trial.params['x'])
            points_y.append(trial.params['y'])
            trial_indices.append(i) # 记录 trial 在 trials 列表中的索引

    # 使用颜色渐变映射
    if trial_indices and len(trial_indices) > 0: # 确保有采样点才进行颜色映射
        # print('max(trial_indices)', max(trial_indices))
        normalized_indices = np.array(trial_indices) / max(trial_indices ) if trial_indices else np.array([]) # 归一化 trial 索引到 0-1 范围
        point_colors = cm.get_cmap(cmap_points)(normalized_indices) # 使用 cmap_points 获取颜色

        # 绘制采样点，使用颜色映射 (cmap_points 用于点颜色)
        colors = ['red', 'blue', 'green'] # 仍然保留 sampler 的颜色，但可以考虑移除或调整
        markers = ['o', 'x', '^']
        sampler_index = list(samplers.keys()).index(sampler_name) % len(markers)
        plt.scatter(points_x, points_y, c=point_colors, marker=markers[sampler_index], s=20, label=sampler_name) # 使用 c=point_colors
    else: # 如果没有采样点，仍然绘制空的 scatter，保持图例
        colors = ['red', 'blue', 'green']
        markers = ['o', 'x', '^']
        sampler_index = list(samplers.keys()).index(sampler_name) % len(markers)
        plt.scatter([], [], color=colors[sampler_index], marker=markers[sampler_index], s=20, label=sampler_name)


    

    # # 获取到当前 trial_number 为止的采样点
    # trials = study.trials[:trial_number] # 获取前 trial_number 个 trials
    # points_x = [t.params['x'] for t in trials if t.state == optuna.trial.TrialState.COMPLETE] # 确保只绘制成功的 trials
    # points_y = [t.params['y'] for t in trials if t.state == optuna.trial.TrialState.COMPLETE]

    # # 绘制采样点
    # colors = ['red', 'blue', 'green'] # 为不同 Sampler 设置颜色 (这里只用了前3个，如果更多sampler需要扩展颜色列表)
    # markers = ['o', 'x', '^']
    # sampler_index = list(samplers.keys()).index(sampler_name) # 获取当前 sampler 的索引
    # plt.scatter(points_x, points_y, color=colors[sampler_index], marker=markers[sampler_index], s=20, label=sampler_name)

    plt.title(f"{sampler_name} - Optuna Search Process (Trial {trial_number})\nBest Value: {best_value_so_far:.4f}") # Formatted to 4 decimal places
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend(loc='upper right') # 图例放在右上角
    plt.grid(False) # 可以选择不显示网格线
    return plt.gcf() # 返回当前的 figure 对象

# 5. 生成动画
def animate_sampler_search(sampler_name, sampler, n_trials=100, interval=200, repeat_delay=1000, cmap_points='viridis', cmap_contour='winter'): # 添加 cmap_points 参数
    """为指定的 Sampler 生成动画视频"""
    study = optuna.create_study(sampler=sampler)

    best_value_history = [] # To store best value after each trial
    current_best_value = float('inf') # Initialize with positive infinity for minimization

    # 存储每个 trial 的结果，以便在动画中使用
    x_history = []
    y_history = []
    value_history = []

    def animated_objective(trial): # 包装 objective 函数，记录历史数据
        nonlocal current_best_value # Allow modification of outer scope variable
        x = trial.suggest_float('x', -5, 5)
        y = trial.suggest_float('y', -5, 5)
        value = ackley_2d(x, y)
        x_history.append(x)
        y_history.append(y)
        value_history.append(value)

        if value < current_best_value: # For minimization, if maximizing, change to '>'
            current_best_value = value
        best_value_history.append(current_best_value) # Store best value after this trial

        return value

    study.optimize(animated_objective, n_trials=n_trials, show_progress_bar=True)

    print('study', study)

    fig = plt.figure(figsize=(10, 8)) # 创建 figure 对象，所有帧都将绘制在这个 figure 上

    def update_frame(frame_number):
        best_value_for_frame = best_value_history[frame_number] # Get best value for this frame
        return create_frame(frame_number + 1, study, sampler_name, best_value_for_frame, cmap_points=cmap_points, cmap_contour=cmap_contour) # 传递 cmap_points

    ani = animation.FuncAnimation(fig, update_frame, frames=n_trials, interval=interval, blit=False, repeat_delay=repeat_delay) # 创建动画

    # 保存为 MP4 视频 (需要安装 ffmpeg 或 Pillow writers)
    save_filename = f"optuna_search_{sampler_name}_{n_trials}_{cmap_contour}_{cmap_points}.mp4"
    ani.save(save_filename, writer='ffmpeg', fps=10) # fps 设置帧率，可以调整
    # 或者保存为 GIF (如果安装了 Pillow writers)
    # ani.save(f"optuna_search_{sampler_name}.gif", writer='pillow', fps=15)

    plt.close(fig) # 关闭 figure，释放内存
    print(f"动画已保存为 {save_filename}")


# 6. 循环生成不同 Sampler 的动画
for sampler_name, sampler in samplers.items():
    for cmap_points in color_map_options:
        for n_trails in n_trails_tries:
            animate_sampler_search(sampler_name, sampler, n_trials=n_trails, interval=0, cmap_points=cmap_points) # 循环尝试不同颜色映射

print("所有 Sampler 的动画生成完成!")